In [ ]:
import os
os.getcwd()

'd:\\Digital-Twin-Systems-for-Rumor-Analysis\\src'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# Load the dataset
file_path = '/Digital-Twin-Systems-for-Rumor-Analysis/datasets/AnnotatedDataset.csv'
df = pd.read_csv(file_path)

df.head()

,text,is_rumor,user.handle,topic,annotation
0,Charlie Hebdo became well known for publishing...,0.0,BBCDanielS,charliehebdo,True
1,"Now 10 dead in a shooting there today RT ""@BBC...",0.0,robbylevy,charliehebdo,True
2,@BBCDanielS @BBCWorld I'm guessing this is bei...,0.0,ModerateInAll,charliehebdo,True
3,@BBCDanielS @BBCWorld why would you mention th...,0.0,GabTarquini,charliehebdo,True
4,@BBCDanielS @BBCWorld perps identified?,0.0,freethought41,charliehebdo,True


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49666 entries, 0 to 49665
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   text         49666 non-null  object 
 1   is_rumor     49666 non-null  float64
 2   user.handle  49666 non-null  object 
 3   topic        49666 non-null  object 
 4   annotation   49666 non-null  bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 1.6+ MB


In [ ]:
stance_mapping = {
    # Support Labels
    'true': 'support',
    'agreed': 'support',
    True: 'support',  # Added boolean True

    # Deny Labels
    'false': 'deny',
    'disagreed': 'deny',
    False: 'deny', # Added boolean False

    # Query Labels
    'appeal-for-more-information': 'query',

    # Comment Labels (for general discussion or unverified claims)
    'unverified': 'comment',
    'comment': 'comment',
    'discuss': 'comment'
}

# --- 3. Apply the Mapping ---
# Create a new 'stance' column based on the 'annotation' column.
df['stance'] = df['annotation'].map(stance_mapping)
print("Applied stance mapping to create a new 'stance' column.")

# --- 4. Verification ---
# Check if any annotations were not mapped. This is important for data quality.
unmapped_count = df['stance'].isnull().sum()
if unmapped_count > 0:
    print(f"\nWARNING: {unmapped_count} annotations could not be mapped.")
    unmapped_values = df[df['stance'].isnull()]['annotation'].unique()
    print("The following unique values were not found in the mapping dictionary:")
    print(unmapped_values)
    print("Please add them to the 'stance_mapping' dictionary and rerun the script.")
else:
    print("\nSUCCESS: All annotation labels were mapped successfully.")

# Display the first 15 rows to show the original annotation and the new stance
print("\n--- Sample of the processed data ---")
print(df[['text', 'annotation', 'stance']].head(15))
print("------------------------------------")


# --- 5. Save the Processed Data ---
# Save the updated DataFrame to a new CSV file for the next steps.
output_filename = '/Digital-Twin-Systems-for-Rumor-Analysis/datasets/dataset_with_stances.csv'
# df.to_csv(output_filename, index=False)
print(f"\nProcessed data has been saved to '{output_filename}'.")


Applied stance mapping to create a new 'stance' column.

SUCCESS: All annotation labels were mapped successfully.

--- Sample of the processed data ---
                                                 text  annotation   stance
0   Charlie Hebdo became well known for publishing...        True  support
1   Now 10 dead in a shooting there today RT "@BBC...        True  support
2   @BBCDanielS @BBCWorld I'm guessing this is bei...        True  support
3   @BBCDanielS @BBCWorld why would you mention th...        True  support
4             @BBCDanielS @BBCWorld perps identified?        True  support
5         @BBCDanielS @BBCWorld who is charlie hebdo?        True  support
6          @im_a_5H_Voter A French satirical magazine        True  support
7   @GabTarquini @BBCDanielS @BBCWorld Maybe becau...        True  support
8   @S_Jakobsen @BBCDanielS @BBCWorld what's your ...        True  support
9   @GabTarquini @BBCDanielS @BBCWorld Several men...        True  support
10  @GabTarquini @BBCDa

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

stance = pd.read_csv('/Digital-Twin-Systems-for-Rumor-Analysis/datasets/dataset_with_stances.csv')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if not isinstance(text, str):
        return 'neutral'

    scores = analyzer.polarity_scores(text)
    compound_score = scores['compound']

    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

print("Applying sentiment analysis... This may take a moment for a large dataset.")
stance['sentiment'] = stance['text'].fillna('').apply(get_sentiment)
print("Sentiment analysis complete.")

print("\n--- Sample of the processed data with sentiment labels ---")
print(stance[['text', 'stance', 'sentiment']].head(15))
print("----------------------------------------------------------")

output_filename = '/Digital-Twin-Systems-for-Rumor-Analysis/datasets/dataset_with_stances.csv'
# stance.to_csv(output_filename, index=False)
print(f"\nProcessed data has been saved to '{output_filename}'.")
print("This file is now ready for the next step: structuring conversations.")


Applying sentiment analysis... This may take a moment for a large dataset.
Sentiment analysis complete.

--- Sample of the processed data with sentiment labels ---
                                                 text   stance sentiment
0   Charlie Hebdo became well known for publishing...  support  positive
1   Now 10 dead in a shooting there today RT "@BBC...  support  negative
2   @BBCDanielS @BBCWorld I'm guessing this is bei...  support  negative
3   @BBCDanielS @BBCWorld why would you mention th...  support   neutral
4             @BBCDanielS @BBCWorld perps identified?  support   neutral
5         @BBCDanielS @BBCWorld who is charlie hebdo?  support   neutral
6          @im_a_5H_Voter A French satirical magazine  support   neutral
7   @GabTarquini @BBCDanielS @BBCWorld Maybe becau...  support   neutral
8   @S_Jakobsen @BBCDanielS @BBCWorld what's your ...  support   neutral
9   @GabTarquini @BBCDanielS @BBCWorld Several men...  support   neutral
10  @GabTarquini @BBCDanielS @BBC

In [ ]:
sentiment = pd.read_csv('/Digital-Twin-Systems-for-Rumor-Analysis/datasets/dataset_with_stances.csv')
sentiment.head()

,text,is_rumor,user.handle,topic,annotation,stance,sentiment
0,Charlie Hebdo became well known for publishing...,0.0,BBCDanielS,charliehebdo,True,support,positive
1,"Now 10 dead in a shooting there today RT ""@BBC...",0.0,robbylevy,charliehebdo,True,support,negative
2,@BBCDanielS @BBCWorld I'm guessing this is bei...,0.0,ModerateInAll,charliehebdo,True,support,negative
3,@BBCDanielS @BBCWorld why would you mention th...,0.0,GabTarquini,charliehebdo,True,support,neutral
4,@BBCDanielS @BBCWorld perps identified?,0.0,freethought41,charliehebdo,True,support,neutral


In [ ]:
print("Grouping posts by topic to form conversation threads...")
conversations = sentiment.groupby('topic')

structured_conversations = {}

for topic, group in conversations:
    posts = group.to_dict('records')

    if posts:
        posts[0]['is_source_rumor'] = True
        for post in posts[1:]:
            post['is_source_rumor'] = False

    structured_conversations[topic] = posts

print(f"Successfully structured the data into {len(structured_conversations)} conversations.")

if 'charliehebdo' in structured_conversations:
    print("\n--- Example: 'charliehebdo' conversation ---")
    sample_conversation = structured_conversations['charliehebdo']
    print(f"Topic: charliehebdo")
    print(f"Total Posts: {len(sample_conversation)}")
    print("Source Rumor:")
    print(f"  Text: {sample_conversation[0]['text']}")
    print(f"  Stance: {sample_conversation[0]['stance']}")
    print(f"  Sentiment: {sample_conversation[0]['sentiment']}")
    print("First Comment:")
    if len(sample_conversation) > 1:
        print(f"  Text: {sample_conversation[1]['text']}")
        print(f"  Stance: {sample_conversation[1]['stance']}")
        print(f"  Sentiment: {sample_conversation[1]['sentiment']}")
    print("---------------------------------------------")

import json

output_filename = '/Digital-Twin-Systems-for-Rumor-Analysis/datasets/structured_conversations.json'
with open(output_filename, 'w') as f:
    json.dump(structured_conversations, f, indent=4)

print(f"\nStructured conversation data has been saved to '{output_filename}'.")
print("Your data is now ready for the final step: calculating harmfulness scores.")


Grouping posts by topic to form conversation threads...
Successfully structured the data into 2 conversations.

--- Example: 'charliehebdo' conversation ---
Topic: charliehebdo
Total Posts: 25491
Source Rumor:
  Text: Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago
  Stance: support
  Sentiment: positive
First Comment:
  Text: Now 10 dead in a shooting there today RT "@BBCDanielS: Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago”
  Stance: support
  Sentiment: negative
---------------------------------------------

Structured conversation data has been saved to '/Digital-Twin-Systems-for-Rumor-Analysis/datasets/structured_conversations.json'.
Your data is now ready for the final step: calculating harmfulness scores.
